In [ ]:
import os
from dotenv import load_dotenv
import requests
import pandas as pd
from datetime import datetime, timedelta, timezone
from openai import AzureOpenAI

# Load environment variables
load_dotenv()

github_token = os.getenv("VSCODE_INSTRUCTIONS")
pr_number = os.getenv("PR_NUMBER")
endpoint = "https://models.inference.ai.azure.com/"

if github_token is None or not github_token.strip():
	raise EnvironmentError("Environment variable VSCODE_INSTRUCTIONS is missing or empty.")

if pr_number is None or not pr_number.strip():
	raise EnvironmentError("Environment variable PR_NUMBER is missing or empty.")

headers = {
    "Authorization": f"Bearer {github_token}",
    "Accept": "application/vnd.github.v3+json",
}

client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=github_token,
    api_version="2025-01-01-preview",
)

print(f"Processing PR #{pr_number}")

# instruction_prompt = "Read the uncommitted changes in the workspace and understand what has been changed, then read all the instructions files under the `.github/instructions` folder and see if the changes you made invalidate any of the instructions. If so, provide a proposal to users on how to update the instructions to reflect the changes you made. If no updates are needed, just say \"No updates needed\". Be concise and conservative with your suggestions. Only suggest changes that are absolutely necessary."

response = client.chat.completions.create(
	messages=[
		# {"role": "system", "content": system_prompt},
		# {"role": "user", "content": instruction_prompt},
		{"role": "user", "content": "hello"}
	],
	model="gpt-4.1",
	temperature=0,
	max_tokens=1000,
	top_p=1,
)

ai_response = response.choices[0].message.content
print(f"AI Response: {ai_response}")

# Post comment to PR
def post_pr_comment(pr_number, comment_body, github_token):
    """Post a comment to a GitHub PR"""
    url = f"https://api.github.com/repos/microsoft/vscode/issues/{pr_number}/comments"

    headers = {
        "Authorization": f"Bearer {github_token}",
        "Accept": "application/vnd.github.v3+json",
        "Content-Type": "application/json"
    }

    payload = {
        "body": comment_body
    }

    response = requests.post(url, headers=headers, json=payload)

    if response.status_code == 201:
        print(f"Successfully posted comment to PR #{pr_number}")
        return response.json()
    else:
        print(f"Failed to post comment. Status: {response.status_code}")
        print(f"Response: {response.text}")
        response.raise_for_status()

# Format the comment with the AI response
comment_body = f"""## Instruction Update Analysis

{ai_response}

---
*This comment was automatically generated by the GitHub Actions workflow.*
"""

# Post the comment to the PR
try:
    post_pr_comment(pr_number, comment_body, github_token)
except Exception as e:
    print(f"Error posting comment: {e}")
    raise

Hello! How can I help you today?
